# Setup the Environment
We are going to use YOLOv5 model for training.
so lets clone and install the YOLO from github repo

!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import xml.etree.ElementTree as ET 
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
input_data = r"C:/Users/visha/OneDrive/Desktop/vishal/yolov5/dataset"
output_data = r"C:\Users\visha\OneDrive\Desktop\vishal"

# Read and Handle Annotations
Read the data from .xml files
<br>
Extract filename and dimensions of the bouding boxes
<br>
and import that data into a pandas DataFrame

In [ ]:
annotations_path = r"C:\Users\visha\OneDrive\Desktop\vishal\yolov5\dataset\annotations"
images_path = r"C:\Users\visha\OneDrive\Desktop\vishal\yolov5\dataset\images"

In [ ]:
dataset = {
            "file":[],
            "name":[],    
            "width":[],
            "height":[],
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
           }

for anno in glob.glob(annotations_path+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['name']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]

In [ ]:
df=pd.DataFrame(dataset)
df.head()

In [ ]:
for idx,i in enumerate(df['name']):
    if i =='mask_weared_incorrect' or i == 'with_mask':
        df['name'][idx] = 'Mask_Detected'
    else:
        if i=='without_mask':
            df['name'][idx] = 'No_Mask'
            
            

In [ ]:

df.head()

In [ ]:
name_dict = {
    'Mask_Detected': 0,
    'No_Mask': 1
}

df['class'] = df['name'].map(name_dict)

In [ ]:
np.sort(df.name.unique())

# Train Test Split
Split the dataset into training testing and validation set
<br>
separate the data in different folder inside the Yolo directory
<br>
and Normalize data

In [ ]:
fileNames = [*os.listdir(r"C:\Users\visha\OneDrive\Desktop\vishal\yolov5\dataset\images")]
print('There are {} images in the dataset'.format(len(fileNames)))

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(fileNames, test_size=0.1, random_state=22)
test, val = train_test_split(test, test_size=0.7, random_state=22)
print("Length of Train =",len(train))
print("="*30)
print("Length of Valid =",len(val))
print("="*30)
print("Length of test =", len(test))

In [ ]:
# create separate train test and validation dir inside yolo/data/
os.chdir(r"C:\Users\visha\OneDrive\Desktop\vishal")
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\train')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\val')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\test')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\train\images')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\train\labels')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\test\images')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\test\labels')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\val\images')
os.mkdir(r'C:\Users\visha\OneDrive\Desktop\vishal\yolov5\data\val\labels')

In [ ]:

from PIL import Image

def copyImages(imageList, folder_Name):
    for image in imageList:
        img = Image.open(input_data+"/images/"+image)
        img1 = img.resize((640, 480))
        _ = img1.save(output_data+"/yolov5/data/"+folder_Name+"/images/"+image)
print(train)
copyImages(train, "train")
copyImages(val, "val")
copyImages(test, "test")

In [ ]:
df.head()

In [ ]:
df['xmax'] = (640/df['width'])*df['xmax']
df['ymax'] = (480/df['height'])*df['ymax']
df['xmin'] = (640/df['width'])*df['xmin']
df['ymin'] = (480/df['height'])*df['ymin']

In [ ]:
df[['xmax', 'ymax', 'xmin', 'ymin']] = df[['xmax', 'ymax', 'xmin', 'ymin']].astype('int64')

In [ ]:
df['x_center'] = (df['xmax']+df['xmin'])/(2*640)
df['y_center'] = (df['ymax']+df['ymin'])/(2*480)
df['box_height'] = (df['xmax']-df['xmin'])/(640)
df['box_width'] = (df['ymax']-df['ymin'])/(480)

In [ ]:
df.iloc[5]

In [ ]:
ff = glob.glob(r"C:/Users/visha/Downloads/archive/images**/*.png",recursive=True)

In [ ]:
len(ff)

!pip install pytest-shutil

In [ ]:
import shutil

In [ ]:
for i in ff:  
    g =i.split('\\')[-1]
    if g.split('.')[0] in fileNames:
        print('dd')
        shutil.copy(i, r"C:/Users/visha/OneDrive/Desktop/vishal/yolov5/data/train/images/")
        

In [ ]:
fileNames

In [ ]:
df = df.astype('string')

In [ ]:
fileNames = [x.split(".")[0] for x in train]

In [ ]:
len(fileNames)

In [ ]:
fileNames

In [ ]:
for i in fileNames:
    
    print(f'{i}.txt')

In [ ]:
def create_labels(image_list, data_name):
    fileNames = [x.split(".")[0] for x in image_list]

    for name in fileNames:
        data = df[df.file==name]
        box_list = []
        
        for index in range(len(data)):
            row = data.iloc[index]
            box_list.append(row['class']+" "+row["x_center"]+" "+row["y_center"]\
                        +" "+row["box_height"]+" "+row["box_width"])
            
        text = "\n".join(box_list)
        with open(output_data+"/yolov5/data/"+data_name+"/labels/"+name+".txt", "w") as file:
            file.write(text)


create_labels(train, "train")
create_labels(val, "val")
create_labels(test, "test")

# Configure the Model

In [ ]:
# go to YOLO directory
%cd yolov5

!pip install yolov5

In [ ]:
# to display images
from IPython.display import Image, clear_output
import torch
from yolov5 import utils
display = utils.notebook_init()

In [ ]:
# configure .yaml file to guide the model for training
yaml_text = """train: data/train/images
val: data/train/images

nc: 2
names: ['No_Mask', 'Mask_Detected']"""

with open("data/data.yaml", 'w') as file:
    file.write(yaml_text)

%cat data/data.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: 2  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
    - [10,13, 16,30, 33,23]  # P3/8
    - [30,61, 62,45, 59,119]  # P4/16
    - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
    [[-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
    [[-1, 6], 1, Concat, [1]],  # cat backbone P4
    [-1, 3, BottleneckCSP, [512, False]],  # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 'nearest']],
    [[-1, 4], 1, Concat, [1]],  # cat backbone P3
    [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]],  # cat head P4
    [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]],  # cat head P5
    [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
    ]

In [ ]:
os. getcwd()

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
# we are going to using pre-trained weights from yolov5.pt model

start = datetime.now()
!python train.py --img 640 --batch 32 --epochs 50 --data data/data.yaml --cfg models/custom_yolov5s.yaml --weights yolov5s.pt --name yolov5s_results  --cache
end = datetime.now()

In [ ]:
print("Runtime =",end-start)

In [ ]:
# lets visualize the training results
img = plt.imread('runs/train/yolov5s_results/train_batch0.jpg')
plt.figure(figsize=(20,15))
plt.imshow(img)
plt.axis('off')
plt.show()

# Test the Model
lets test the model over test images we had separated before training

In [23]:
label ='jj'
a ='22'

In [28]:
ee  = f'{label}   {a}'

In [65]:
!python detect.py --source video/222.mp4 --weight runs/train/exp/weights/best_300.pt --conf 0.5

detect: weights=['runs/train/exp/weights/best_300.pt'], source=video/222.mp4, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-30-g342fe05 Python-3.9.13 torch-1.13.0+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7249215 parameters, 0 gradients
video 1/1 (1/1703) C:\Users\visha\OneDrive\Desktop\vishal\yolov5\video\222.mp4: 384x640 1 No_Mask, 485.5ms
video 1/1 (2/1703) C:\Users\visha\OneDrive\Desktop\vishal\yolov5\video\222.mp4: 384x640 1 No_Mask, 408.6ms
video 1/1 (3/1703) C:\Users\visha\OneDrive\Desktop\vishal\yolov5\video\222.mp4: 384x640 1 No_Mask, 426.2ms
video 1/1 (4/1703) C:\Users\visha\OneDrive\Desktop\vishal\yo

runs\detect\exp27\222.mp4
1
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
runs\detect\exp27\222.mp4
25
det[i] tensor([458.00000, 268.00000, 634.00000, 468.00000,   0.74681,   1.00000])
det[j][0] tensor(458.)
det[i] tensor([458.00000, 268.00000, 634.00000, 468.00000,   0.74681,   1.00000])
det[j][0] tensor(901.)
det[i] tensor([9.01000e+02, 2.04000e+02, 1.08700e+03, 4.61000e+02, 5.45865e-01, 1.00000e+00])
det[j][0] tensor(458.)
det[i] tensor([9.01000e+02, 2.04000e+02, 1.08700e+03, 4.61000e+02, 5.45865e-01, 1.00000e+00])
det[j][0] tensor(901.)
det[i] tensor([458.00000, 268.00000, 634.00000, 468.00000,   0.74681,   1.00000])
det[j][0] tensor(458.)
det[i] tensor([458.00000, 268.00000, 634.00000, 468.00000,   0.74681,   1.00000])
det[j][0] tensor(901.)
det[i] tensor([9.01000e+02, 2.04000e+02, 1.08700e+03, 4.6

In [ ]:
!python detect.py --source 0 --weight runs/train/exp/weights/best_300.pt --conf 0.5

results are saved inside run/detect/expTestImage dir <br>
let's check the results to see how our model performed

In [ ]:
color_dict = {
    'with_mask': (0, 255, 0),
    'mask_weared_incorrect':  (0, 0, 255),
    'without_mask': (255, 0, 0) 
}

In [ ]:
def show_image(img_id):
    df_image = df[df.file==img_id]
    df_image[['xmin', 'ymin', 'xmax', 'ymax']] = df_image[['xmin', 'ymin', 'xmax', 'ymax']].astype('int64')
    path = 'data/test/images/'+img_id# +'.png'
    img = plt.imread(path)

    imge = img.copy()

    for index in range(len(df_image)):
        row = df_image.iloc[index]
        cv2.rectangle(imge, 
                      (row['xmin'], row['ymin']),
                      (row['xmax'], row['ymax']),
                      color=color_dict[row['name']],
                      thickness=2)

    img_pred = plt.imread('runs/detect/expTestImage/'+img_id)
    # ===================================
    plt.figure(figsize=(14,17))

    plt.subplot(1,2,1)
    plt.imshow(imge)
    plt.axis('off')
    plt.title('Image with Truth Box')

    plt.subplot(1,2,2)
    plt.imshow(img_pred)
    plt.axis('off')
    plt.title('Image with Predicted Box')

In [ ]:
import os, random
show_image(random.choice(os.listdir("data/test/images/"))) 
show_image(random.choice(os.listdir("data/test/images/"))) 
plt.show()

# END